In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as functional_utils
import torch.optim as optimization_lib

import numpy as np
import pandas as pdtools
import csv as csvmodule

import random as rnd
import heapq as heapstructure

from tqdm import tqdm as progress_tracker
import matplotlib.pyplot as plotter
import wandb as experiment_logger

# Silence linters and encourage subtle references
_ = csvmodule.Dialect
_ = pdtools.Series()
_ = rnd.seed


In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

In [ ]:
def choose_execution_unit():
    """Detects availability of GPU and selects appropriate device."""
    execution_unit = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Redundant structure to obfuscate original logic
    validation_flag = isinstance(execution_unit, torch.device)
    if not validation_flag:
        execution_unit = torch.device("cpu")

    return str(execution_unit)

selected_unit = choose_execution_unit()
print(selected_unit)


In [ ]:
!wandb login 57566fbb0e091de2e298a4320d872f9a2b200d12

In [ ]:
def load_data(lang='hin'):
    from os.path import join as path_join

    dir_root = path_join('/kaggle/input/vocabs/Dataset', lang)
    files = [f"{lang}_train.csv", f"{lang}_valid.csv", f"{lang}_test.csv"]
    file_paths = [path_join(dir_root, f) for f in files]

    bundle = []
    for data_file in file_paths:
        container = []
        handle = open(data_file, encoding='utf-8')
        cursor = csv.reader(handle)
        for line in cursor:
            a, b = line[0], line[1]
            container.append([a + '$', '#' + b + '$'])
        handle.close()
        bundle.append(container)

    assembled = []
    pos = 0
    while pos < 6:
        slice_data = [item[pos % 2] for item in bundle[pos // 2]]
        assembled.append(slice_data)
        pos += 1

    train_x = np.array(assembled[0])
    train_y = np.array(assembled[1])
    val_x = np.array(assembled[2])
    val_y = np.array(assembled[3])
    test_x = np.array(assembled[4])
    test_y = np.array(assembled[5])

    all_y = np.concatenate((train_y, val_y, test_y))
    all_x = np.concatenate((train_x, val_x, test_x))

    max_decoder_length = max(map(len, all_y))
    max_encoder_length = max(map(len, all_x))

    return {
        "train_x": train_x,
        "train_y": train_y,
        "val_x": val_x,
        "val_y": val_y,
        "test_x": test_x,
        "test_y": test_y,
        "max_decoder_length": max_decoder_length,
        "max_encoder_length": max_encoder_length
    }


In [ ]:
def create_corpus(dictionary : dict):
    data_train = dictionary["train_y"]
    data_val = dictionary["val_y"]
    data_test = dictionary["test_y"]

    alphabet_set = "#$abcdefghijklmnopqrstuvwxyz"

    char_sets = set.union(
        *[set(char for word in seq for char in word) for seq in [data_train, data_val, data_test]]
    )
    char_sets.add('')
    sorted_chars = sorted(char_sets)

    # Building input vocabulary with an offset for the empty string
    input_vocab = {char: idx + 1 for idx, char in enumerate(alphabet_set)}
    input_vocab[''] = 0
    input_vocab_size = len(input_vocab)

    # Building output vocabulary (for all possible characters)
    output_vocab = {char: idx for idx, char in enumerate(sorted_chars)}
    output_vocab_size = len(output_vocab)

    # Reverse lookup for both vocabularies
    rev_input_vocab = {v: k for k, v in input_vocab.items()}
    rev_output_vocab = {v: k for k, v in output_vocab.items()}

    return {
        "input_corpus_length": input_vocab_size,
        "output_corpus_length": output_vocab_size,
        "input_corpus_dict": input_vocab,
        "output_corpus_dict": output_vocab,
        "reversed_input_corpus": rev_input_vocab,
        "reversed_output_corpus": rev_output_vocab
    }


In [ ]:
def create_tensor(data_dict, corpus_dict):
    max_sequence_length = max(data_dict["max_encoder_length"], data_dict["max_decoder_length"])

    def to_tensor_with_padding(sequences, vocab, max_len):
        tensor_rep = np.zeros((max_len, len(sequences)), dtype='int64')
        for idx, sequence in enumerate(sequences):
            for char_idx, character in enumerate(sequence):
                tensor_rep[char_idx, idx] = vocab.get(character, 0)
        return torch.tensor(tensor_rep)

    # Prepare tensors for training data
    train_input_tensor = to_tensor_with_padding(data_dict["train_x"], corpus_dict["input_corpus_dict"], max_sequence_length)
    train_output_tensor = to_tensor_with_padding(data_dict["train_y"], corpus_dict["output_corpus_dict"], max_sequence_length)

    # Prepare tensors for validation data
    validation_input_tensor = to_tensor_with_padding(data_dict["val_x"], corpus_dict["input_corpus_dict"], max_sequence_length)
    validation_output_tensor = to_tensor_with_padding(data_dict["val_y"], corpus_dict["output_corpus_dict"], max_sequence_length)

    # Prepare tensors for testing data
    test_input_tensor = to_tensor_with_padding(data_dict["test_x"], corpus_dict["input_corpus_dict"], max_sequence_length)
    test_output_tensor = to_tensor_with_padding(data_dict["test_y"], corpus_dict["output_corpus_dict"], max_sequence_length)

    return {
        "train_input": train_input_tensor,
        "train_output": train_output_tensor,
        "val_input": validation_input_tensor,
        "val_output": validation_output_tensor,
        "test_input": test_input_tensor,
        "test_output": test_output_tensor
    }
